In [1]:
import numpy as np # for processing of arrays
import pandas as pd
import sklearn # to display model performance on test set
import statistics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt # to display images from dataset
import os
from glob import glob
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from tensorflow.keras.applications.efficientnet import EfficientNetB4
# import tensorflow backend and keras api
import tensorflow as tf
import keras
import keras.backend as K

# import model layers and InceptionV3 architecture
from tensorflow.python.keras.models import Model
from sklearn.ensemble import RandomForestClassifier

# import optimizers and callbacks
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.layers import VersionAwareLayers

layers = VersionAwareLayers()
Dropout = layers.Dropout
Dense = layers.Dense
Input = layers.Input
concatenate = layers.concatenate
GlobalAveragePooling2D = layers.GlobalAveragePooling2D
AveragePooling2D = layers.AveragePooling2D
Flatten = layers.Flatten

base_model1 = EfficientNetB4(include_top=False, weights="imagenet", input_tensor=Input(shape=(299,299,3)))
layer1 = GlobalAveragePooling2D()(base_model1.output)

layer1 = Dense(2, activation="softmax", name="output")(layer1)
input_layer = base_model1.input
model1 = Model(inputs=input_layer, outputs=layer1, name="EN4")


# display model summary
model1.summary()

71688192/71686520 [==============================] - 1s 0us/step
Model: "EN4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 299, 299, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 301, 301, 3)  0           normalization[0][0]              
_______________________________

In [2]:
print("GPU:", tf.config.list_physical_devices('GPU'), "\nCUDA Enabled:", tf.test.is_built_with_cuda(), "\nGPU Name:", tf.test.gpu_device_name(), "\nVisible Devices:", tf.config.experimental.list_physical_devices('GPU'))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 
CUDA Enabled: True 
GPU Name: /device:GPU:0 
Visible Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3940348040546067378
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15685569792
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16108603391565410559
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [9]:
# ImageDataGenerator loads images into memory in batches of specified size (in this case 16 images per batch)
# this avoids possible memory issues
train_folder = '../input/deep-fake-images/train'
val_folder = '../input/deep-fake-images/val'
df_train = pd.read_csv(train_folder + '/image_labels.csv')
df_val = pd.read_csv(val_folder + '/image_labels.csv')

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,horizontal_flip=True, vertical_flip=True, width_shift_range=0.10, rotation_range=0.2) # rescaling normalizes pixel values from the range [0,255] to [0,1]
train_set = datagen.flow_from_dataframe(dataframe=df_train, directory=train_folder, classes= ['real', 'fake'], class_mode="categorical", target_size=(299, 299), batch_size=32)
val_set = datagen.flow_from_dataframe(dataframe=df_val, directory=val_folder, classes= ['real', 'fake'], class_mode="categorical", target_size=(299, 299), batch_size=32)

print("Check class name mapping to label index:")
print(train_set.class_indices)
print(val_set.class_indices)

Found 66722 validated image filenames belonging to 2 classes.
Found 12592 validated image filenames belonging to 2 classes.
Check class name mapping to label index:
{'fake': 0, 'real': 1}
{'fake': 0, 'real': 1}


In [10]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.001, amsgrad=True)

In [6]:
model1.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
checkpoint = ModelCheckpoint("./EB4.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [12]:
hist = model1.fit(train_set, steps_per_epoch=64, epochs = 40, validation_data=val_set, validation_steps=32, callbacks = [checkpoint])

Epoch 1/40
64/64 [==============================] - 44s 669ms/step - loss: 0.4439 - accuracy: 0.8311 - val_loss: 0.6934 - val_accuracy: 0.8457

Epoch 00001: val_loss did not improve from 0.44108
Epoch 2/40
64/64 [==============================] - 42s 662ms/step - loss: 0.5115 - accuracy: 0.7861 - val_loss: 0.4862 - val_accuracy: 0.8164

Epoch 00002: val_loss did not improve from 0.44108
Epoch 3/40
64/64 [==============================] - 43s 673ms/step - loss: 0.4648 - accuracy: 0.8232 - val_loss: 0.4716 - val_accuracy: 0.8242

Epoch 00003: val_loss did not improve from 0.44108
Epoch 4/40
64/64 [==============================] - 43s 665ms/step - loss: 0.4947 - accuracy: 0.7979 - val_loss: 0.4493 - val_accuracy: 0.8320

Epoch 00004: val_loss did not improve from 0.44108
Epoch 5/40
64/64 [==============================] - 43s 665ms/step - loss: 0.4696 - accuracy: 0.8154 - val_loss: 0.4612 - val_accuracy: 0.8301

Epoch 00005: val_loss did not improve from 0.44108
Epoch 6/40
64/64 [=======

In [13]:
model1.load_weights("./EB4.h5")

In [14]:
def read_image_from_disk(path):

  """
  Helper function to read image from disk given a absolute path.

  :param path: Absolute path to image file on disk
  :return: Image in Numpy Ndarray representation
  """

  img = tf.keras.preprocessing.image.load_img(path, target_size=(299,299,3))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = img/255
  img = np.expand_dims(img, axis=0)

  return img


def get_frames_to_vid_mapping(frame_list):

  """
  Helper function to generate a mapping of frames to it's corresponding video 
  name.

  The path of frames in the frame_list will be in such format:
  image/[video name]/[frame number].jpg
  e.g. image/00000/00032.jpg

  :param frame_list: A list of paths to the image frames
  :return: A sorted dictionary with keys as the video name and value as the
           corresponding frames.
           e.g. of returned mapping dictionary:

            {
              "00000":[
                  "00032",
                  "00064",
                  .
                  .
                  .
                  "00487"
              ],
              "00001":[
                  "00000",
                  "00032",
                  .
                  .
                  .
                  "00392"
              ],
              .
              .
              .
              "00790":[
                  "00000",
                  "00027",
                  .
                  .
                  .
                  "00542"
              ]
            }
  """

  # Get all videos name
  vidnames = [frame.split("/")[1:2][0] for frame in frame_list]
  # Get only unique names
  vidnames = set(vidnames)
  # Init the mapping dict
  mapping = {vidname: [] for vidname in vidnames}

  # Add frames to to its corresponding list
  for frame in frame_list:
    vidname = str(frame.split("/")[1:2][0])
    frame_number = str(frame.split("/")[-1].split(".")[0])
    mapping[vidname].append(frame_number)

  return dict(sorted(mapping.items()))


def infer_videos(test_data_path, csv_file, num_of_videos='All'):

  """
  Function to infer a test data set. The function takes in a path to the test
  data set and a csv file that contain the paths of the frames extracted from 
  the videos in the test dataset.

  :param test_data_path: Absoulute path to the test dataset
  :param csv_file: File Name of the CSV file that must be in the test_data_path
  :param num_of_videos: Number of videos to infer from the dataset (default: All)
  :return: Pandas dataframe which contains the prediction (probability of being 
           fake) of each video. 
  """

  list_dir = list(pd.read_csv(test_data_path + csv_file).iloc[:,0])

  mapping = get_frames_to_vid_mapping(list_dir)

  # [*mapping] gives the list of keys (video name) in the mapping dict
  num_of_videos_avail = len([*mapping])

  # Set number of videos to be inferred to total of videos available if given 
  # num_of_videos is more than max amount of available videos
  if num_of_videos == 'All' or num_of_videos > num_of_videos_avail:
      num_of_videos = num_of_videos_avail

  # init mapping of videos to its corresponding predicted probabilities
  videos_to_prediction = {}

  # Loop through each video and make a prediction of each frame in the video.
  # Assigned a prediction to each video by taking the mean of its corresponding
  # frames' probabilities.
  for video_name in [*mapping][0:num_of_videos]:

    frames = mapping[video_name]
    predictions = []
    print("Infering video {video}...".format(video=video_name))
    print("Processing frame ", end=" ")

    # Process each frame in video
    for frame in frames:
      print(frame, end =", ")
      frame_path = "image/{video_name}/{frame}.jpg".format(video_name=video_name, frame=frame)
      img = read_image_from_disk(test_data_path + frame_path)
      prediction = model.predict(img)[0]
      # Collect only the 'real' side of probability
      predictions.append(prediction[1])

    # Take the mean of the probabilities from the frames
    videos_to_prediction[video_name] = statistics.median(predictions)
    print("Done!")
  
  return pd.DataFrame(videos_to_prediction.items())


In [15]:
model = model1
modelPredictions = infer_videos("../input/deep-fake-images/test/", "image_labels.csv")
print(modelPredictions)

Infering video 00000...
Processing frame  00032, 00064, 00097, 00129, 00162, 00194, 00227, 00259, 00292, 00324, 00357, 00389, 00422, 00454, 00487, Done!
Infering video 00001...
Processing frame  00000, 00032, 00065, 00098, 00130, 00163, 00196, 00228, 00261, 00294, 00326, 00359, 00392, Done!
Infering video 00002...
Processing frame  00000, 00027, 00054, 00081, 00108, 00135, 00162, 00189, 00216, 00243, 00271, 00298, 00325, 00352, 00379, 00406, 00433, 00460, 00487, 00514, 00542, Done!
Infering video 00003...
Processing frame  00000, 00031, 00063, 00095, 00127, 00159, 00191, 00222, 00254, 00286, 00318, 00350, 00382, 00413, 00445, 00477, 00509, 00541, 00573, 00605, Done!
Infering video 00004...
Processing frame  00000, 00026, 00052, 00078, 00105, 00131, 00157, 00183, 00210, 00236, 00262, 00289, 00315, 00341, 00367, 00394, 00420, 00446, 00472, 00499, 00525, 00551, 00578, 00604, 00630, 00656, 00683, 00709, 00735, 00762, Done!
Infering video 00005...
Processing frame  00000, 00025, 00051, 0007

In [17]:
modelPredictions.columns = ['vid_name', 'label']
modelPredictions.to_csv("./Puah Yi Hao_yihao.puah@gmail.com.csv", index=False)